<h1 style = "font-size: 35px; text-align: center;">AI Genetic Hands On</h1>
<h1 style = "font-size: 25px; text-align: center;">Hospital Job Scheduling for Doctors</h1>
<h1 style = "font-size: 25px; text-align: center; color: #666">Taha Shabani - Soheil Shirvani</h1>
<h4 style="text-align: center">Spring 1400</h4>

In [45]:
testFile = "test1.txt"

In [46]:
def readInput() :
    file = open(testFile, 'r+')
    fileList = file.readlines()
    fileList = [s.replace('\n', '') for s in fileList]
    
    [days, doctors] = [int(i) for i in fileList[0].split()]
    maxCapacity = int(fileList[1])
    offDays = int(fileList[2])
    
    allShifts = []
    for i in range(3, days + 3):
        dayRequirements = fileList[i].split()
        morningReqs = [int(i) for i in dayRequirements[0].split(",")]
        eveningReqs = [int(i) for i in dayRequirements[1].split(",")]
        nightReqs = [int(i) for i in dayRequirements[2].split(",")]
        
        allShifts.append((morningReqs, eveningReqs, nightReqs))
        
#     print(days, doctors, maxCapacity, offDays)
#     for i in allShifts:
#         print(i.printInfo())

    file.close()
    return [days, list(range(doctors)), maxCapacity, offDays, allShifts];

days, doctorsIds, maxCapacity, offDays, allShifts = readInput()

In [55]:
import numpy as np
from random import sample, randrange, shuffle, uniform

def sortSecond(val): 
    return val[1]

class JobScheduler:
    def __init__(self, fileInfo):
        self.days = fileInfo[0]
        self.doctors = len(fileInfo[1])
        self.doctorsIds = fileInfo[1]
        self.maxCapacity = fileInfo[2]
        self.offDays = fileInfo[3]
        self.allShifts = fileInfo[4]
        
        self.maxFitness = 100
        

        self.restartLimitation = 10000 #consideration: if we stuck in local maximums this would be needed
        self.popSize = 250
        self.crossOverPoints = 10
        self.mutationPoints = 1
        self.elitismPercentage = 4 #move x% best of parents directly to the new population
        self.pc = 0.6 #crossover probability
        self.pm = 0.4  #mutation probability
        
        self.chromosomeSet = {} #[0]: chromosome hash to fitness value
        self.chromosomes = self.getInitialChromosomes() #[0]: chromosome value, [1]: fitness value
        
        self.chromosomes.sort(key = sortSecond, reverse = True)
#         display(self.chromosomes)
#         display(self.chromosomes[0])
#         display(self.chromosomes[1])
#         temps = self.crossOver(self.chromosomes[0], self.chromosomes[1])
#         display(temps[0])
#         display(temps[1])
        
#         display(self.chromosomes[0])
#         self.mutate(0)
#         display(self.chromosomes[0])
#         self.calculateFitness(self.chromosomes[0])
        
        
    
    
    def getInitialChromosomes(self):
        population = []
        for i in range(self.popSize):
            newChrome = self.createChromosome()
            population.append(newChrome)
            self.chromosomeSet[self.getChromeHash(newChrome[0])] = newChrome[1]
            
        return population
    
    def createChromosome(self):
        allShiftIds = [[0 for i in range(3)] for j in range(self.days)]

        for day in range(self.days):
            [morningDoctors, eveningDoctors, nightDoctors] = sample(list(range(1,self.doctors)), 3)
            
            allShiftIds[day][0] = sample(list(range(self.doctors)), morningDoctors)
            allShiftIds[day][1] = sample(list(range(self.doctors)), eveningDoctors)
            allShiftIds[day][2] = sample(list(range(self.doctors)), nightDoctors)

        return [allShiftIds, self.calculateFitness(allShiftIds)]
    
    def getChromeHash(self, chromeVal):
        hashString = ""
        for day in range(len(chromeVal)):
            for shift in range(3):
                for did in chromeVal[day][shift]:
                    hashString += (str(did) + ",")
                hashString += "/"
            hashString += "|"
        return hashString
    
    def crossOver(self, chrome1, chrome2):
        chromeVal1 = chrome1[0]
        chromeVal2 = chrome2[0]
        
        points = sorted(sample(list(range(self.days*3)), self.crossOverPoints))
        for i in points:
            if uniform(0, 1) <= self.pc:
#                 print("hey: ", i)
                temp = chromeVal1[int(i/3)][i%3]
                chromeVal1[int(i/3)][i%3] = chromeVal2[int(i/3)][i%3]
                chromeVal2[int(i/3)][i%3] = temp
                
        newChrome1 = [chromeVal1, self.calculateFitness(chromeVal1)]
        newChrome2 = [chromeVal2, self.calculateFitness(chromeVal2)]
        
        return [newChrome1, newChrome2]
        
        
                
    def mutate(self, chrome):
        chromeVal = chrome[0]
        for i in range(self.mutationPoints):
            if uniform(0, 1) <= self.pm:
                [i, j] = sample(list(range(self.days*3)), 2)        
                temp = chromeVal[int(i/3)][i%3]
                chromeVal[int(i/3)][i%3] = chromeVal[int(j/3)][j%3]
                chromeVal[int(j/3)][j%3] = chromeVal[int(i/3)][i%3]

    #             print("hoy: ", randomDay)
    #             [morningDoctors, eveningDoctors, nightDoctors] = sample(list(range(self.doctors + 1)), 3)

    #             chromeVal[randomDay][0] = sample(list(range(self.doctors)), morningDoctors)
    #             chromeVal[randomDay][1] = sample(list(range(self.doctors)), eveningDoctors)
    #             chromeVal[randomDay][2] = sample(list(range(self.doctors)), nightDoctors)
            
        newChrome = [chromeVal, self.calculateFitness(chromeVal)]
        return newChrome
            
    def calculateFitness(self, chromeVal):
        chromeHash = self.getChromeHash(chromeVal)
        if (chromeHash in self.chromosomeSet):
            return self.chromosomeSet[chromeHash]
        
        nightShiftViolations = self.calculateNightShiftViolations(chromeVal)
        minMaxReqsViolations = self.calculateMinMaxReqsViolations(chromeVal)
        maxShiftViolations = self.calculateMaxShiftViolations(chromeVal)
        totalCost =  nightShiftViolations +  maxShiftViolations
        
        self.chromosomeSet[chromeHash] = self.maxFitness - totalCost
        return self.maxFitness - totalCost
    
    def calculateNightShiftViolations(self, chromeVal):
        violations = 0
        
        #morning or evening shift after a night shift
        for day in range(len(chromeVal)-1):
            for nightDoctor in chromeVal[day][2]:
                if (nightDoctor in chromeVal[day+1][0]):
                    violations += 1
                if (nightDoctor in chromeVal[day+1][1]):
                    violations += 1
        
        #three night shifts in a row
        for day in range(len(chromeVal)-2):
            for nightDoctor in chromeVal[day][2]:
                if (nightDoctor in chromeVal[day+1][2] and nightDoctor in chromeVal[day+2][2]):
                    violations += 1
        
        return violations
    
    def calculateMinMaxReqsViolations(self, chromeVal):
        violations = 0
        
        for day in range(len(chromeVal)):
            for shift in range(3):
                shiftDoctors = len(chromeVal[day][shift])
                if (shiftDoctors < allShifts[day][shift][0] or shiftDoctors > allShifts[day][shift][1]):
                    violations += 1
        
        return violations
    
    def calculateMaxShiftViolations(self, chromeVal):
        violations = 0
        doctorShifts = [0 for i in self.doctorsIds]
        
        for day in range(len(chromeVal)):
            for shift in range(3):
                for did in chromeVal[day][shift]:
                    doctorShifts[did] = doctorShifts[did] + 1
                    
        for shift in doctorShifts:
            if shift > maxCapacity:
                violations += 1
        
        return violations
    
    def getNewParent(self, sumOfFitnesses) : 
        randNum = randrange(sumOfFitnesses)
        sumOfFitnesses = 0
        for i in range(self.popSize) : 
            sumOfFitnesses += self.chromosomes[i][1]
            if sumOfFitnesses >= randNum : 
                return self.chromosomes[i]
            
        index = randrange(self.popSize)
        return self.chromosomes[index]  
    
    def chooseParents(self):
        parents = []
        elitism = int(self.elitismPercentage/100 * self.popSize)
        sumOfFitnesses = 0
        for i in range(self.popSize) : 
            sumOfFitnesses += self.chromosomes[i][1]
        for i in range(self.popSize - elitism) : 
            newParent = self.getNewParent(sumOfFitnesses)
            parents.append(newParent)
        shuffle(parents)
        return parents
    
    def mateParentsAndGetChildren(self, parents) :
        newGeneration = []
        elitism = int(self.elitismPercentage/100 * self.popSize)
        for i in range(elitism) : 
            newGeneration.append(self.chromosomes[i])
        for i in range(int((self.popSize - elitism) / 2)) :
            parent1 = parents[(i * 2) % (self.popSize)]
            parent2 = parents[(i * 2 + 1) % (self.popSize)]
            newChildren = self.crossOver(parent1, parent2)
            child1 = self.mutate(newChildren[0])
            child2 = self.mutate(newChildren[1])
            newGeneration.append(child1)
            newGeneration.append(child2)   
        newGeneration.sort(key = sortSecond, reverse = True)
        return newGeneration
    
    def generateNewGeneration(self):
        newParents = self.chooseParents()
        self.chromosomes = self.mateParentsAndGetChildren(newParents)
    
    def schedule(self):
        currentBestFitness = self.chromosomes[0][1]
        iteration = 0
        repeated = 0
        
#         for i in range(self.popSize):
#                 display(self.chromosomes[i])
#         print()
        
        while (currentBestFitness < self.maxFitness):
            
            self.generateNewGeneration()
            
            if (self.chromosomes[0][1] == currentBestFitness) :
                repeated += 1
            else: 
                repeated = 0
            if (repeated == self.restartLimitation) :
                repeated = 0
                print("restart................................................................")
                self.chromosomes = self.getInitialChromosomes()
                self.chromosomes.sort(key = sortSecond, reverse = True)    
            
            currentBestFitness = self.chromosomes[0][1]
                      
            for i in range(3):
                print(self.chromosomes[i][1], end=",")
            print()
        
            iteration += 1
        display(self.chromosomes[0])

In [ ]:
import time

fileInfo = readInput()

start = time.time()

scheduler = JobScheduler(fileInfo)
scheduler.schedule()

end = time.time()

print("time: ", '%.2f'%(end - start), 'sec')

90,89,89,
94,93,93,
94,94,93,
94,94,93,
94,94,93,
94,94,93,
94,94,93,
94,94,93,
94,94,93,
94,94,93,
94,94,93,
94,94,94,
94,94,94,
94,94,94,
94,94,94,
96,94,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
96,96,94,
